# Importando libs e dataset


In [10]:
!pip install scikit_posthocs

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from pathlib import Path
from itertools import combinations

from scipy.stats import friedmanchisquare
import scikit_posthocs as sp # Para os testes post-hoc

# aumentar o range de um texto nas colunas
pd.set_option('display.max_colwidth', None)

In [12]:
results_path = Path("results_table")
results_path.exists()

True

In [13]:
rq2_table_path = results_path / 'rq2_formatted.xlsx'
df = pd.read_excel(rq2_table_path)
df.columns

Index(['Model', 'bitter frustration', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'impatience', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'vulgarity', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45'],
      dtype='object')

In [14]:
rq2_table_bitter = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Bitter Frustration')
rq2_table_impatience = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Impatience')
rq2_table_vulgarity = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Vulgarity')
rq2_table_irony = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Irony')
rq2_table_ia = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Identity Attack')
rq2_table_threat = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Threat')
rq2_table_insulting = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Insulting')
rq2_table_entitlement = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Entitlement')
rq2_table_mocking = pd.read_excel(rq2_table_path, index_col=0, sheet_name='Mocking')
rq2_table_none = pd.read_excel(rq2_table_path, index_col=0, sheet_name='None')

rq2_list = [rq2_table_bitter,rq2_table_impatience,rq2_table_vulgarity,rq2_table_irony,rq2_table_ia,rq2_table_threat,rq2_table_insulting,rq2_table_entitlement,rq2_table_mocking,rq2_table_none]

In [15]:
rq2_table_bitter.columns

Index(['Zero-shot', 'Zero-shot.1', 'Zero-shot.2', 'One-shot', 'One-shot.1',
       'One-shot.2', 'Few-shot', 'Few-shot.1', 'Few-shot.2', 'Auto-CoT',
       'Auto-CoT.1', 'Auto-CoT.2', 'Role-based', 'Role-based.1',
       'Role-based.2'],
      dtype='object')

# Realizando teste de friedman para fine-grained

In [16]:
import pandas as pd
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

# Lista de classes e estratégias
classes = ['Bitter Frustration', 'Impatience', 'Vulgarity', 'Irony',
           'Identity Attack', 'Threat', 'Insulting', 'Entitlement', 'Mocking', 'None']
strategies = ['Zero-shot.2', 'One-shot.2', 'Few-shot.2', 'Auto-CoT.2', 'Role-based.2']
alpha = 0.05

# Dicionário para armazenar os dados limpos
f1_data = {}

# Preprocessamento e organização dos dados
for i, classe in enumerate(classes):
    df = rq2_list[i].replace(',', '.', regex=True)
    df[strategies] = df[strategies].astype(float)
    df.drop('Average', axis=0, inplace=True)
    f1_data[classe] = df[strategies]

# Função para executar o teste de Friedman e post-hoc de Nemenyi
def run_friedman_analysis(data, class_name, strategy_labels, alpha):
    print(f"\n📊 Análise para a classe: {class_name}")

    # Aplica o teste de Friedman
    stat, p_value = friedmanchisquare(*data.T.values)
    print(f"Friedman: chi² = {stat:.4f}, p = {p_value:.4f}")

    if p_value < alpha:
        print("🔍 Diferença significativa detectada. Realizando teste de Nemenyi...")
        nemenyi = sp.posthoc_nemenyi_friedman(data.values)
        nemenyi.columns = strategy_labels
        nemenyi.index = strategy_labels
        print("\n📌 Matriz de p-valores (teste de Nemenyi):")
        # print(nemenyi.round(4))

        print("\n🚩 Estratégias com diferença significativa (p < 0.05):")
        significant_pairs = []
        for i in range(len(strategy_labels)):
            for j in range(i+1, len(strategy_labels)):
                p = nemenyi.iloc[i, j]
                if p < alpha:
                    print(f"- {strategy_labels[i]} vs {strategy_labels[j]} (p = {p:.4f})")
                    significant_pairs.append((strategy_labels[i], strategy_labels[j]))
        if not significant_pairs:
            print("Nenhuma diferença significativa detectada entre pares.")
    else:
        print("✅ Não há diferença estatisticamente significativa entre as estratégias.")

# Executa a análise para todas as classes
for classe in classes:
    run_friedman_analysis(f1_data[classe], classe, strategies, alpha)



📊 Análise para a classe: Bitter Frustration
Friedman: chi² = 21.0400, p = 0.0003
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
- Zero-shot.2 vs Few-shot.2 (p = 0.0377)
- One-shot.2 vs Role-based.2 (p = 0.0377)
- Few-shot.2 vs Auto-CoT.2 (p = 0.0160)
- Few-shot.2 vs Role-based.2 (p = 0.0004)

📊 Análise para a classe: Impatience
Friedman: chi² = 22.7200, p = 0.0001
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
- Zero-shot.2 vs One-shot.2 (p = 0.0160)
- One-shot.2 vs Few-shot.2 (p = 0.0002)
- One-shot.2 vs Auto-CoT.2 (p = 0.0377)
- Few-shot.2 vs Role-based.2 (p = 0.0160)

📊 Análise para a classe: Vulgarity
Friedman: chi² = 16.1818, p = 0.0028
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):


In [17]:
import pandas as pd
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

# Lista de classes e estratégias
classes = ['Bitter Frustration', 'Impatience', 'Vulgarity', 'Irony',
           'Identity Attack', 'Threat', 'Insulting', 'Entitlement', 'Mocking', 'None']
strategies = ['Zero-shot', 'One-shot', 'Few-shot', 'Auto-CoT', 'Role-based']
alpha = 0.05

# Dicionário para armazenar os dados limpos
pr_data = {}

# Preprocessamento e organização dos dados
for i, classe in enumerate(classes):
    df = rq2_list[i].replace(',', '.', regex=True)
    df[strategies] = df[strategies].astype(float)
    df.drop('Average', axis=0, inplace=True)
    pr_data[classe] = df[strategies]

# Executa a análise para todas as classes
for classe in classes:
    run_friedman_analysis(pr_data[classe], classe, strategies, alpha)



📊 Análise para a classe: Bitter Frustration
Friedman: chi² = 22.9600, p = 0.0001
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
- Zero-shot vs One-shot (p = 0.0377)
- One-shot vs Few-shot (p = 0.0000)
- Few-shot vs Role-based (p = 0.0377)

📊 Análise para a classe: Impatience
Friedman: chi² = 20.7200, p = 0.0004
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
- Zero-shot vs One-shot (p = 0.0062)
- One-shot vs Few-shot (p = 0.0013)
- Few-shot vs Role-based (p = 0.0377)

📊 Análise para a classe: Vulgarity
Friedman: chi² = 19.1472, p = 0.0007
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
- Zero-shot vs Few-shot (p = 0.0127)
- Few-shot vs 

In [18]:
for classe in classes:
    run_friedman_analysis(f1_data[classe].T, classe, f1_data[classe].index, alpha)


📊 Análise para a classe: Bitter Frustration
Friedman: chi² = 21.9646, p = 0.0090
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
Nenhuma diferença significativa detectada entre pares.

📊 Análise para a classe: Impatience
Friedman: chi² = 17.7449, p = 0.0383
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
Nenhuma diferença significativa detectada entre pares.

📊 Análise para a classe: Vulgarity
Friedman: chi² = 27.0256, p = 0.0014
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
- gemma2:9b vs gpt-4o-mini (p = 0.0468)
- llama3.2:3b vs gpt-4o-mini (p = 0.0202)

📊 Análise para a classe: Irony
Friedman: chi² = 23.9961, p = 0.0043
🔍 Diferença 

In [19]:
for classe in classes:
    run_friedman_analysis(pr_data[classe].T, classe, pr_data[classe].index, alpha)


📊 Análise para a classe: Bitter Frustration
Friedman: chi² = 17.6127, p = 0.0399
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
- phi4:14b vs mistral:7b (p = 0.0286)

📊 Análise para a classe: Impatience
Friedman: chi² = 17.2105, p = 0.0455
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
Nenhuma diferença significativa detectada entre pares.

📊 Análise para a classe: Vulgarity
Friedman: chi² = 18.4615, p = 0.0302
🔍 Diferença significativa detectada. Realizando teste de Nemenyi...

📌 Matriz de p-valores (teste de Nemenyi):

🚩 Estratégias com diferença significativa (p < 0.05):
Nenhuma diferença significativa detectada entre pares.

📊 Análise para a classe: Irony
Friedman: chi² = 20.0652, p = 0.0175
🔍 Diferença significativa detectada. Realizando teste de